In [1]:
import pickle
import random
import numpy as np

import torch
import torch.nn as nn
from model import Actor, Critic, DRRAveStateRepresentation, PMF

In [2]:
state_rep_net = DRRAveStateRepresentation(n_items=5, item_features=100, user_features=100)
actor_net = Actor(in_features=300, out_features=100)

In [3]:
def init_weights(m):
    if hasattr(m, 'weight'):
        nn.init.orthogonal_(m.weight.data)
    if hasattr(m, 'bias'):
        nn.init.constant_(m.bias.data, 0)

In [4]:
state_rep_net.apply(init_weights)

DRRAveStateRepresentation()

In [5]:
actor_net.apply(init_weights)

Actor(
  (linear1): Linear(in_features=300, out_features=300, bias=True)
  (linear2): Linear(in_features=300, out_features=300, bias=True)
  (linear3): Linear(in_features=300, out_features=100, bias=True)
)

In [6]:
state_rep_net.load_state_dict(torch.load('results/220620-122520/state_rep_net.weights'))

<All keys matched successfully>

In [7]:
actor_net.load_state_dict(torch.load('results/220620-122520/actor_net.weights'))

<All keys matched successfully>

In [8]:
def seed_all(cuda, seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.empty_cache()
        torch.cuda.manual_seed(seed=seed)

no_cuda = True
cuda = True if not no_cuda and torch.cuda.is_available() else False
print("Using CUDA") if cuda else print("Using CPU")
 
 
# Init seeds
seed_all(cuda, 0)
print("Seeds initialized")

Using CPU
Seeds initialized


In [9]:
train_ratio = 0.8

# Import Data
users = pickle.load(open('dataset/user_id_to_num.pkl', 'rb'))
items = pickle.load(open('dataset/rest_id_to_num.pkl', 'rb'))
data = np.load('dataset/data.npy')

np.random.shuffle(data)
train_data = torch.from_numpy(data[:int(train_ratio * data.shape[0])])
test_data = torch.from_numpy(data[int(train_ratio * data.shape[0]):])
print("Data imported, shuffled, and split into Train/Test, ratio=", train_ratio)
print("Train data shape: ", train_data.shape)
print("Test data shape: ", test_data.shape)

Data imported, shuffled, and split into Train/Test, ratio= 0.8
Train data shape:  torch.Size([83880, 4])
Test data shape:  torch.Size([20970, 4])


In [10]:
test_data.device

device(type='cpu')

In [11]:
from utils.history_buffer import HistoryBuffer

In [12]:
history_buffer = HistoryBuffer(5)

In [13]:
history_buffer

In [14]:
user_idxs = np.array(list(users.values()))
np.random.shuffle(user_idxs)

In [15]:
len(user_idxs)

1245

In [16]:
n_users = len(users)
n_items = len(items)
reward_function = PMF(n_users, n_items, 100, is_sparse=False, no_cuda=~cuda)
reward_function.load_state_dict(torch.load('trained/yelp_ratio_0.800000_bs_1024_e_93_wd_0.100000_lr_0.000100_trained_pmf.pt'))
 
# Freeze all the parameters in the network
for param in reward_function.parameters():
    param.requires_grad = False
print("Initialized PMF, imported weights, created reward_function")
 
# Extract embeddings
user_embeddings = reward_function.user_embeddings.weight.data
item_embeddings = reward_function.item_embeddings.weight.data
print("Extracted user and item embeddings from PMF")
print("User embeddings shape: ", user_embeddings.shape)
print("Item embeddings shape: ", item_embeddings.shape)

Initialized PMF, imported weights, created reward_function
Extracted user and item embeddings from PMF
User embeddings shape:  torch.Size([1245, 100])
Item embeddings shape:  torch.Size([40829, 100])


In [17]:
device = torch.device('cpu')

In [18]:
candidate_item_idxs = np.arange(item_embeddings.shape[0])
candidate_item_idxs = torch.from_numpy(candidate_item_idxs).to(device).long()

In [19]:
candidate_item_idxs

tensor([    0,     1,     2,  ..., 40826, 40827, 40828])

In [20]:
e = 1058
user_reviews = test_data[test_data[:, 0] == e]
print(user_reviews.shape)
pos_user_reviews = user_reviews[user_reviews[:, 2] > 3]
print(pos_user_reviews.shape)

torch.Size([152, 4])
torch.Size([62, 4])


In [21]:
user_reviews = user_reviews[user_reviews[:, 3].sort()[1]]
pos_user_reviews = pos_user_reviews[pos_user_reviews[:, 3].sort()[1]]

In [22]:
candidate_items = item_embeddings.detach().clone().to(device)
user_candidate_items = item_embeddings[user_reviews[:, 1]].detach().clone().to(device)

In [23]:
user_candidate_items.shape

torch.Size([152, 100])

In [24]:
for i in range(5):
    emb = candidate_items[pos_user_reviews[i, 1]]
    history_buffer.push(emb.detach().clone())

In [25]:
user_embed = user_embeddings[e]

In [26]:
state = state_rep_net(user_embed, torch.stack(history_buffer.to_list()))

In [27]:
state.shape

torch.Size([300])

In [28]:
action = actor_net(state.detach())

/Users/ankurdhuriya/miniconda3/envs/drr_pytorch/lib/python3.10/site-packages/torch/nn/functional.py:1933: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [29]:
action.shape

torch.Size([100])

In [30]:
candidate_items.shape

torch.Size([40829, 100])

In [31]:
ranking_scores = candidate_items @ action

In [32]:
rec_item_idx = torch.argmax(ranking_scores)
rec_item_emb = candidate_items[rec_item_idx]

In [33]:
rec_item_idx

tensor(1928)

In [34]:
reward_function(torch.tensor(e).to(device), rec_item_idx)

tensor(6.2708)

In [35]:
values, indices = torch.topk(ranking_scores, 10)

In [36]:
values, indices

(tensor([3.9093, 3.7243, 3.6243, 3.3859, 3.3565, 3.3095, 3.2925, 3.2847, 3.2625,
         3.2567], grad_fn=<TopkBackward0>),
 tensor([ 1928, 27426, 34728, 22638, 30189, 14687, 23123, 39257, 33578, 15967]))

In [37]:
for i in indices:
    rec_item_emb = candidate_items[i]
    reward = reward_function(torch.tensor(e).to(device), i)
    print(reward)

tensor(6.2708)
tensor(6.3561)
tensor(6.3335)
tensor(6.0134)
tensor(6.3228)
tensor(6.1564)
tensor(5.7862)
tensor(6.0286)
tensor(5.9696)
tensor(6.2333)


In [40]:
latest_item_embeds = torch.stack(history_buffer.to_list())

In [41]:
candidate_items[1928].shape

torch.Size([100])

In [44]:
cos = nn.CosineSimilarity()

In [119]:
inv_users = {v: k for k, v in users.items()}

In [123]:
inv_items = {v: k for k, v in items.items()}

In [126]:
import pandas as pd

In [127]:
df = pd.read_csv('../data/yelp-restaurants/yelp_review.csv')

In [130]:
df[df.business_id==inv_items[1928]]

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
5971,nUKp1aFIrLSCXuNzAuwiRQ,cvzKVWaGFysF1iZ1RUvsBg,1xO2LvScpcMfl2BNm1aISg,5,2009-09-01,As I said I had no idea what was going on for ...,0,0,0
9030,EsIKolmy5TAfWtsSmPplBQ,oH-pRxI7Q-qZbZaoVQHeTw,1xO2LvScpcMfl2BNm1aISg,5,2017-11-28,This is PARADISE!\nI've been here for a prenat...,1,0,0
11581,wHPyNtxcTILqM3aC2axLNA,MW5yJ20UFL2zpmajGP37JA,1xO2LvScpcMfl2BNm1aISg,5,2012-10-07,A solid 4.5 stars (I always round up). \n\nIf ...,6,1,1
18824,f_p_n4xjhyryYKK94iwB8A,tilVfA96HboviIrz80wQsg,1xO2LvScpcMfl2BNm1aISg,5,2015-02-21,Loved it again! This was my second visit and c...,1,0,0
32224,Il7hniY1jxfpSEHcUupycg,JAJCM-LpEyw5zEsYvjrjqg,1xO2LvScpcMfl2BNm1aISg,3,2017-05-13,"The spa services are still great, but my exper...",2,1,0
...,...,...,...,...,...,...,...,...,...
5211007,pQEUum91-8-gojnwfwK47A,RY4LTB9IBbGsMyp-0F9klw,1xO2LvScpcMfl2BNm1aISg,5,2014-12-08,Relax to the max! This is probably the best s...,1,0,0
5213014,GkIgElOn6Zl4AJjcW-UjmQ,DbMYj86AsnDpm-gMVbr5jg,1xO2LvScpcMfl2BNm1aISg,2,2014-09-28,Not impressed. I found it odd that they had a ...,4,0,0
5231920,-lZYnobSSbZtMiXFzb7lkQ,-GBkfd3ETbKgwrv2D-lNrg,1xO2LvScpcMfl2BNm1aISg,5,2017-09-29,This was one of the few spas I could find that...,5,9,1
5253696,CVEw7jf3zO-936Bc-nqCNw,JCPMFtvErIR-SsqZGU71xA,1xO2LvScpcMfl2BNm1aISg,5,2010-04-09,"Favorite spa in Vegas, been to most of them. ...",3,0,1


In [132]:
df[df.user_id==inv_users[1058]]

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
2955910,Y6TbOcAahwUd2ovIVAwY9g,pMefTWo6gMdx8WhYSA2u3w,BQsHIeY0x__B5IHyyQZ1ng,3,2017-11-20,A friend of mine pointed out this restaurant t...,2,0,0
2955911,cPySMRBCs0_jhZUT8DEOpA,pMefTWo6gMdx8WhYSA2u3w,BjH8Xepc10i6OhCDQdX6og,5,2015-03-22,This is one of the restaurants located in the ...,5,1,3
2955912,ObX7eD5iPTJq2tbxjXzh1Q,pMefTWo6gMdx8WhYSA2u3w,e3a_sCmg2E0MKwLDLJ3q5Q,3,2013-08-21,"Hard to rank variety stores..it has gum, it ha...",0,1,0
2955913,vJQNO97GcUv89f1oC0-j5g,pMefTWo6gMdx8WhYSA2u3w,MmWYE3hKBn0LW24V2TT9Iw,4,2013-10-14,This used to be Timothy's way back when. Last ...,0,0,0
2955914,bXu6dNtZiCOf7gw7qpJPBg,pMefTWo6gMdx8WhYSA2u3w,bsGVvCtj8i2xGl15XX8Dxw,3,2013-12-08,Ordered burrito bowl. Meat was fresh and every...,0,0,0
...,...,...,...,...,...,...,...,...,...
3679779,zkmAyAv0SNHKBHu3YgSArw,pMefTWo6gMdx8WhYSA2u3w,Yo_cFLv1Kf92LP1JSwjDmw,4,2014-03-14,Brand new Hakka restaurant that opened up just...,2,0,0
3679780,G7SVVUe1um1AntQUARjUgQ,pMefTWo6gMdx8WhYSA2u3w,7YYrZ9LgjpKLTtF-huhJug,4,2013-09-18,"Okay, I've seen some reviews where people comp...",2,0,3
3679781,sRIY-HeNltHFn7ZJJD0eVw,pMefTWo6gMdx8WhYSA2u3w,E0h1kK-8P_DtBHk5ClVI_g,2,2012-09-05,I want to give this place higher and may in a ...,1,0,1
3679782,gF6iwrTqT0vQ4ND_Gcp35g,pMefTWo6gMdx8WhYSA2u3w,fY5P4pZEuSM-7qdMAjj4XQ,3,2015-10-17,I was down in Port Credit last night. I order...,4,0,1
